In [1]:
import pandas as pd
from clickhouse_driver import Client
from datetime import datetime
import random
import logging

def read_clickhouse_to_dataframe(host,port,user,password,database):
    query = 'SELECT email_address,name_suffix,gender,name_prefix,birth_date,model_nbr,serial_nbr,first_name,middle_name,last_name,address_1,postal_code,city,state,primary_phone_nbr FROM registria'
    # Connect to ClickHouse server
    client1 = Client(host=host, port=port, user=user, password=password, database=database)
    try:
        # Execute SQL query to fetch data
        result = client1.execute(query)
        columns = ['email_address','name_suffix','gender','name_prefix','birth_date','model_nbr','serial_nbr','first_name','middle_name','last_name','address_1','postal_code','city','state','primary_phone_nbr']
        df = pd.DataFrame(result, columns=columns)
        df=df.to_dict(orient="records")

    finally:
        # Close ClickHouse connection
        client1.disconnect()

    return df

def main_new(dfs,host,port,user,password,database):
    ratings = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    questions= ["On a scale of 0 to 10, how likely are you to recommend XYZ Electronics products to a friend or colleague?","How satisfied are you with the performance of your recent XYZ [specific product] purchase?","How satisfied were you with the customer service you received during your recent interaction with XYZ Electronics?","What improvements, if any, would you suggest for XYZ Electronics products to better meet your needs?","How would you describe your overall perception of the XYZ Electronics brand?","Rate your satisfaction with the ease of the online or in-store purchasing process for your XYZ product."]
    df = read_clickhouse_to_dataframe(host,port,user,password,database)
    lis=[]
    start_percentage = 30
    end_percentage = 100

    total_rows = len(df)
    start_index = int(total_rows * (start_percentage / 100))
    end_index = int(total_rows * (end_percentage / 100))
    df = df[start_index:end_index]
    print(len(df))
    print(len(dfs))
    for i in range(len(df)):
        try:
            birth_date = datetime.strptime(df[i]['birth_date'], '%Y-%m-%d')
            dfs[i]['email_address']=df[i]['email_address']
            dfs[i]['name_prefix']=df[i]['name_prefix']
            dfs[i]['name_suffix']=df[i]['name_suffix']
            dfs[i]['name_first']=df[i]['first_name']
            dfs[i]['name_middle']=df[i]['middle_name']
            dfs[i]['name_last']=df[i]['last_name']
            dfs[i]['birth_month']=birth_date.month
            dfs[i]['birth_year']=birth_date.year
            dfs[i]['gender']=df[i]['gender']
            dfs[i]['address_line_1']=df[i]['address_1']
            dfs[i]['city']=df[i]['city']
            dfs[i]['state']=df[i]['state']
            dfs[i]['zip']=df[i]['postal_code']
            dfs[i]['zip4']=df[i]['postal_code']
            dfs[i]['phone']=df[i]['primary_phone_nbr']
            dfs[i]['model_name']=df[i]['model_nbr']
            dfs[i]['serial_number']=df[i]['serial_nbr']
            dfs[i]['insertedtime']=datetime.now().strftime("%H:%M:%S")
            dfs[i]['location_consent_timestamp']=datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            dfs[i]['email_consent_timestamp']=datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            dfs[i]['phone_consent_timestamp']=datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            dfs[i]['insertion_date']=str(datetime.now().date())
        except Exception as e:
            logging.info(e)
        for j in questions:
            try:
                x={k : v for k,v in dfs[i].items()}

                x['custom_field_4']=j
                x['custom_field_5']=random.choice(ratings)
                # print(id(dfs[i]))

                lis.append(x)
            except Exception as e:
                logging.info(e)
    return lis